# MATH 210 Introduction to Mathematical Computing

## March 9, 2018

* Linear algebra with SciPy
    * Matrix row operations
    * Determinants, tranpose, inverse, trace
    * Eigenvalues and eigenvectors

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from numpy.linalg import matrix_power as mpow
%matplotlib inline

## Linear algebra with SciPy

### Matrix row operations

Last time we defined functions for elementary matrix operations. Let's do one more: `scale_row`.

In [2]:
def add_row(A,k,i,j):
    "Add k times row i to row j in matrix A (using 0 indexing)."
    m = A.shape[0] # The number of rows of A
    E = np.eye(m)
    if i != j:
        E[j,i] = k
    else:
        E[i,j] = k+1
    return E@A

def swap_row(A,i,j):
    "Swap rows i and j in matrix A (using 0 indexing)."
    nrows = A.shape[0] # The number of rows in A
    E = np.eye(nrows)
    E[i,i] = 0
    E[j,j] = 0
    E[i,j] = 1
    E[j,i] = 1
    return E@A

def scale_row(A,k,i):
    "Multiply row i by k in matrix (using 0 indexing)."
    nrows = A.shape[0] # The number of rows in A
    E = np.eye(nrows)
    E[i,i] = k
    return E@A

Let's use our functions to perform Gaussian elimination and solve a linear system of equations $Ax=b$.

In [3]:
A = np.array([[6,15,1],[8,7,12],[2,7,8]])
b = np.array([[2],[14],[10]])

In [4]:
print(A)

[[ 6 15  1]
 [ 8  7 12]
 [ 2  7  8]]


Form the augemented matrix $M$:

In [5]:
M = np.hstack([A,b])
print(M)

[[ 6 15  1  2]
 [ 8  7 12 14]
 [ 2  7  8 10]]


In [6]:
M1 = scale_row(M,1/6,0)
print(M1)

[[  1.           2.5          0.16666667   0.33333333]
 [  8.           7.          12.          14.        ]
 [  2.           7.           8.          10.        ]]


In [7]:
M2 = add_row(M1,-8,0,1)
print(M2)

[[  1.           2.5          0.16666667   0.33333333]
 [  0.         -13.          10.66666667  11.33333333]
 [  2.           7.           8.          10.        ]]


In [8]:
M3 = add_row(M2,-2,0,2)
print(M3)

[[  1.           2.5          0.16666667   0.33333333]
 [  0.         -13.          10.66666667  11.33333333]
 [  0.           2.           7.66666667   9.33333333]]


In [9]:
M4 = scale_row(M3,-1/13,1)
print(M4)

[[ 1.          2.5         0.16666667  0.33333333]
 [ 0.          1.         -0.82051282 -0.87179487]
 [ 0.          2.          7.66666667  9.33333333]]


In [10]:
M5 = add_row(M4,-2,1,2)
print(M5)

[[  1.           2.5          0.16666667   0.33333333]
 [  0.           1.          -0.82051282  -0.87179487]
 [  0.           0.           9.30769231  11.07692308]]


In [11]:
M6 = scale_row(M5,1/M5[2,2],2)
print(M6)

[[ 1.          2.5         0.16666667  0.33333333]
 [ 0.          1.         -0.82051282 -0.87179487]
 [ 0.          0.          1.          1.19008264]]


In [12]:
M7 = add_row(M6,-M6[1,2],2,1)
print(M7)

[[ 1.          2.5         0.16666667  0.33333333]
 [ 0.          1.          0.          0.1046832 ]
 [ 0.          0.          1.          1.19008264]]


In [13]:
M8 = add_row(M7,-M7[0,2],2,0)
print(M8)

[[ 1.          2.5         0.          0.13498623]
 [ 0.          1.          0.          0.1046832 ]
 [ 0.          0.          1.          1.19008264]]


In [14]:
M9 = add_row(M8,-M8[0,1],1,0)
print(M9)

[[ 1.          0.          0.         -0.12672176]
 [ 0.          1.          0.          0.1046832 ]
 [ 0.          0.          1.          1.19008264]]


Success! The solution of $Ax=b$ is

In [15]:
x = M9[:,3]
print(x)

[-0.12672176  0.1046832   1.19008264]


Or, we can do it the easy way...

In [16]:
x = la.solve(A,b)
print(x)

[[-0.12672176]
 [ 0.1046832 ]
 [ 1.19008264]]


### Matrix determinant, inverse, transpose and trace

The subpackage `scipy.linalg` has functions for all the usual linear algebra operations. Let's apply them to our matrix $A$ defined above.

In [17]:
print(A)

[[ 6 15  1]
 [ 8  7 12]
 [ 2  7  8]]


In [18]:
la.det(A)

-725.9999999999999

In [19]:
la.inv(A)

array([[ 0.03856749,  0.15564738, -0.23829201],
       [ 0.05509642, -0.06336088,  0.08815427],
       [-0.05785124,  0.01652893,  0.10743802]])

Notice that we can solve the equation $Ax=b$ using the inverse $x = A^{-1}b$.

In [20]:
Ainv = la.inv(A)
x = Ainv @ b
print(x)

[[-0.12672176]
 [ 0.1046832 ]
 [ 1.19008264]]


But this is a bad idea in general because computing the inverse of a large matrix is computationally expensive and unnecessary!

Let's check that $AA^{-1} = I$.

In [21]:
A @ Ainv

array([[  1.00000000e+00,   1.70002901e-16,  -6.93889390e-17],
       [ -1.11022302e-16,   1.00000000e+00,   0.00000000e+00],
       [ -1.66533454e-16,   8.32667268e-17,   1.00000000e+00]])

We also can compute the transpose $A^T$:

In [22]:
A.T

array([[ 6,  8,  2],
       [15,  7,  7],
       [ 1, 12,  8]])

Or using the function `scipy.linalg.transpose`:

In [23]:
np.transpose(A)

array([[ 6,  8,  2],
       [15,  7,  7],
       [ 1, 12,  8]])

The trace of a square matrix is the sum of the diagonal entries:

In [24]:
np.trace(A)

21

In [25]:
6+7+8

21

### Eigenvalues and eigenvectors

Recall, given a square matrix $A$, a vector $\mathbf{v}$ is an eigenvector for $A$ with eigenvalue $\lambda$ is $A\mathbf{v} = \lambda \mathbf{v}$

Let $A$ be any matrix and consider $S = A A^T$. Then $S$ is a symmetric matrix.

A beautiful result called the [Principal Axis Theorem](https://en.wikipedia.org/wiki/Principal_axis_theorem) states that the eigenvalues of a symmetric matrix are real and the eigenvectors are orthogonal. Let's verify this for $S = AA^T$.

In [26]:
S = A @ A.T

In [27]:
print(S)

[[262 165 125]
 [165 257 161]
 [125 161 117]]


In [28]:
evals, evecs = la.eig(S)

In [29]:
print(evals)

[ 524.98305893+0.j  101.08481987+0.j    9.93212121+0.j]


In [30]:
evals.dtype

dtype('complex128')

Even though the eigenvalues are returned as complex numbers, the imaginary parts are 0 for each eigenvalue and so they are real. We can convert the type to real using the `real` attribute.

In [31]:
evals = evals.real

In [32]:
print(evals)

[ 524.98305893  101.08481987    9.93212121]


The eigenvectors are returned as a matrix where the column at index `i` corresponds to the eigenvalue at index `i` in the 1D NumPy array `evals`.

In [33]:
evecs

array([[ 0.61809779,  0.77885992, -0.10645344],
       [ 0.64796584, -0.58146371, -0.49197584],
       [ 0.44507908, -0.23511099,  0.86407606]])

Let's verify that $Sv_0 = \lambda_0 v_0$ for the eigenvector $v_0$ and eigenvalue $\lambda_0$. Let's compute $(1/\lambda_0)Sv_0$ and see that it's equal to $v_0$.

In [34]:
(1/evals[0]) * S @ evecs[:,0]

array([ 0.61809779,  0.64796584,  0.44507908])

Success! Now let's verify that $\mathbf{v}_0$ is orthogonal to $\mathbf{v}_1$.

In [35]:
np.dot(evecs[:,0],evecs[:,1])

-1.2490009027033011e-16

Success again!